In [25]:
from pydantic import BaseModel, Field
from typing import List
import instructor
from openai import OpenAI
from dotenv import load_dotenv
import contextkit.read as rd

In [26]:
load_dotenv(override=False)

True

## Define a method to get LLM vendor/model prompting best practices


In [27]:
def file2md(url_or_file_path: str) -> str:
    """Convert a file or URL to markdown content.

    If a URL is provided, it fetches the content using contextkit.read.
    If a local file path is provided, it reads the content from the file.

    Parameters
    ----------
    url_or_file_path : str
        Either a URL (starting with 'http://' or 'https://') or a path to a local file.

    Returns
    -------
    str
        The content from the file or URL as a string.
        Returns an error message if the file path is invalid or an error occurs.
    """
    if url_or_file_path.startswith(("http://", "https://")):
        # It's a URL, use contextkit.read
        try:
            return rd.read_url(url_or_file_path, useJina=True)
        except Exception as e:
            return f"Error reading URL {url_or_file_path}: {e}"
    else:
        # It's a file path
        try:
            # Ensure the path is absolute or resolve relative paths
            # This example assumes the file is in the current working directory or a sub-directory
            # For more robust path handling, you might need to adjust this
            # e.g. Path(url_or_file_path).resolve()
            with open(url_or_file_path, "r", encoding="utf-8") as f:
                return rd.read_file(url_or_file_path, useJina=True)
        except FileNotFoundError:
            return f"Error: File not found at {url_or_file_path}"
        except Exception as e:
            return f"Error reading file {url_or_file_path}: {e}"


### Tests


In [28]:
doc = file2md("https://cookbook.openai.com/examples/gpt4-1_prompting_guide")
print(doc)

Title: GPT-4.1 Prompting Guide | OpenAI Cookbook

URL Source: https://cookbook.openai.com/examples/gpt4-1_prompting_guide

Markdown Content:
The GPT-4.1 family of models represents a significant step forward from GPT-4o in capabilities across coding, instruction following, and long context. In this prompting guide, we collate a series of important prompting tips derived from extensive internal testing to help developers fully leverage the improved abilities of this new model family.

Many typical best practices still apply to GPT-4.1, such as providing context examples, making instructions as specific and clear as possible, and inducing planning via prompting to maximize model intelligence. However, we expect that getting the most out of this model will require some prompt migration. GPT-4.1 is trained to follow instructions more closely and more literally than its predecessors, which tended to more liberally infer intent from user and system prompts. This also means, however, that GPT

## Define a way to get the best prompting guide for the selected model (unless a specific document is request)


## Define the response model of your system prompt improver


In [29]:
class SystemPromptImprovement(BaseModel):
    critique: List[str] = Field(
        ..., description="The critique of the original prompt (e.g, what is missing, why it might be inadequate for certain queries)"
    )

    improved_prompt: str = Field(..., description="The improved system prompt.")

    rationale: List[str] = Field(
        ...,
        description="The rationale for the improved prompt you are proposing (e.g., why the improved prompt will better generalize to new queries and potential edge cases)",
    )


## Define a method to build up our system prompt for the improvement task


In [30]:
def build_improver_system_prompt(
    prompt_best_practices_doc: str | None = None,
    example_user_queries: List[str] | None = None,
) -> str:
    system_msg = """\
You are an expert at improving system prompts that are used by LLMs to generate quality responses. 

You will be given an existing system prompt to improve using general prompting best practices. 
If you are provided "Specific LLM Prompt Best Practices" for a particular LLM, prioritize those recommendations over the general ones.

If you are provided examples of user queries, use them to improve the system prompt as well.

## GeneralPrompting Best Practices

---

1. Role and Objective

- Clearly define the persona or role the LLM should adopt and its overall goal. This helps set the stage for the desired behavior.
- Example: “You are an expert technical writer tasked with explaining complex AI concepts to a non-technical audience.”

2. Instructions / Response Rules:

- This is a core component, providing clear, specific, and unambiguous directives for the task. For newer models that interpret instructions literally, it's vital to be explicit about what to do and what not to do.
- Use bullet points or numbered lists for clarity, especially for multiple instructions.
- Examples:
    * “Summarize the following research paper abstract.”
    * “The summary must be exactly three sentences long.”
    * “Avoid using technical jargon above a high-school reading level.”
    * “Do not include any personal opinions or interpretations.”
- For complex instruction sets, consider breaking them into sub-categories (e.g., ### Tone and Style, ### Information to Exclude).

3. Context:

- The relevant background information, data, or text the LLM needs to perform the task. This could be a customer email, a document to summarize, a code snippet to debug, or user dialogue history.
- Example: “[Insert the full text of the customer email here]”
- When providing multiple documents or long context, clear delimiters are crucial.

4. Examples (Few-Shot Prompting):

- Provide one or more examples of desired input-output pairs. This is highly effective for guiding the model towards the correct format, style, and level of detail. Examples can also clarify nuanced instructions or demonstrate complex tool usage.
- Example: Showing one or two sample emails and their ideal bulletpoint action items, or a sample input and the correctly formatted JSON output.
- Ensure that any important behavior demonstrated in your examples is also explicitly stated in your rules/instructions.

5. Reasoning Steps (Inducing Chain-of-Thought):

- For more complex problems, you can instruct the model to “think step by step” or outline a specific reasoning process. This technique, often called Chain-of-Thought (CoT) prompting, encourages the model to break down the problem and can lead to more accurate and well-reasoned outputs, even for models not explicitly trained for internal reasoning.
- Example: “Before generating the summary, first identify the main hypothesis, then list the key supporting evidence, and finally explainthe primary conclusion. Then, write the summary.”

6. Output Formatting Constraints:

- Explicitly define the desired structure, format, or constraints for the LLM’s response. This is critical for programmatic use of the output.
- Examples: 
    * “Respond using only JSON format with the following keys: sender_name (string), main_issue (string), and suggested_action_items (array of strings).” 
    * “Ensure your response is a single paragraph and ends with a question to the user.”

7. Delimiters and Structure:

- Use clear delimiters (e.g., Markdown section headers like ### Instructions ###, triple backticks for code/text blocks, XML tags) to separate different parts of your prompt, such as instructions, context, and examples. This helps the model understand the distinct components of your input, especially in long or complex prompts.
- A general recommended prompt organization, especially for complex prompts or long contexts, is to place overarching instructions or role definitions at the beginning, followed by context and examples, and potentially reiterating key instructions or output format requirements at the end.

---"""

    if prompt_best_practices_doc:
        system_msg += f"\n\n## Specific LLM Prompt Best Practices\n\n---\n\n{prompt_best_practices_doc}\n\n---"
    if example_user_queries:
        system_msg += f"\n\n## Example User Queries\n\n---\n\n{'-\n'.join(example_user_queries)}\n\n---"

    return system_msg

## Define a method to fetch our improved system prompt


In [31]:
def ask_ai(
    existing_system_prompt: str,
    prompt_best_practices_doc: str | None = None,
    example_user_queries: List[str] | None = None,
) -> SystemPromptImprovement:
    # Build the system message for the improver
    improver_sys_msg = build_improver_system_prompt(
        prompt_best_practices_doc=prompt_best_practices_doc,
        example_user_queries=example_user_queries,
    )

    # Get our improved system prompt
    client = instructor.from_openai(OpenAI())
    return client.chat.completions.create(
        model="o4-mini",
        response_model=SystemPromptImprovement,
        messages=[
            {
                "role": "system",
                "content": improver_sys_msg,
            },
            {
                "role": "user",
                "content": f"The existing system prompt:\n{existing_system_prompt}",
            },
        ],
    )

### Tests


In [32]:
STARTING_SYSTEM_PROMPT = """\
You are a friendly and creative culinary assistant specializing in suggesting easy-to-follow recipes.

## Instructions
- Always provide ingredient lists with precise measurements using standard units.
- If the user doesn't specify what ingredients they have available, ask them about their available ingredients rather than assuming what's in their fridge.
- Feel free to suggest common variations or substitutions for ingredients. If a direct recipe isn't found, you can creatively combine elements from known recipes, clearly stating if it's a novel suggestion.
- Always include clear, step-by-step instructions.
- Present only one recipe at a time.

## Rules
- Never suggest recipes that require extremely rare or unobtainable ingredients without providing readily available alternatives.
- Never use offensive or derogatory language.
- If the user indicates they are allergic to any ingredients, do not suggest recipes that include those ingredients. Double check your work!
- If a user asks for a recipe that is unsafe, unethical, or promotes harmful activities, politely decline and state you cannot fulfill that request, without being preachy.

## Output Format
- Structure all your recipe responses clearly using Markdown for formatting.
- Begin every recipe response with the recipe name as a Level 2 Heading (e.g., ## Amazing Blueberry Muffins).
- Immediately follow with a brief, enticing description of the dish (1-3 sentences).
- Next, include a section titled ### Ingredients. List all ingredients using a Markdown unordered list (bullet points).
- Following ingredients, include a section titled ### Instructions. Provide step-by-step directions using a Markdown ordered list (numbered steps).
- Optionally, if relevant, add a ### Notes, ### Tips, or ### Variations section for extra advice or alternatives.

## Examples
---
User: What is a good salmon recipe if I like the skin to be crispy?
Assistant: ## Golden Pan-Fried Salmon

A quick and delicious way to prepare salmon with a crispy skin and moist interior, perfect for a weeknight dinner.

### Ingredients
* 2 salmon fillets (approx. 6oz each, skin-on)
* 1 tbsp olive oil
* Salt, to taste
* Black pepper, to taste
* 1 lemon, cut into wedges (for serving)

### Instructions
1. Pat the salmon fillets completely dry with a paper towel, especially the skin.
2. Season both sides of the salmon with salt and pepper.
3. Heat olive oil in a non-stick skillet over medium-high heat until shimmering.
4. Place salmon fillets skin-side down in the hot pan.
5. Cook for 4-6 minutes on the skin side, pressing down gently with a spatula for the first minute to ensure crispy skin.
6. Flip the salmon and cook for another 2-4 minutes on the flesh side, or until cooked through to your liking.
7. Serve immediately with lemon wedges.

### Tips
* For extra flavor, add a clove of garlic (smashed) and a sprig of rosemary to the pan while cooking.
* Ensure the pan is hot before adding the salmon for the best sear.
---

"""

example_user_queries = [
    "What is the recipe for a chicken parmesan?",
    "What is the recipe for salmon for someone with a nut allergy?",
]

In [33]:
doc = file2md("https://cookbook.openai.com/examples/gpt4-1_prompting_guide")

res = ask_ai(STARTING_SYSTEM_PROMPT, prompt_best_practices_doc=doc, example_user_queries=example_user_queries)
res

SystemPromptImprovement(critique=['Section headings are inconsistent and could be more structured to improve readability and parsing by the model.', 'Clarification strategy is only applied when ingredients are missing; it should more generally prompt for missing context (dietary restrictions, serving size, equipment) when needed.', 'Unit conversion guidance (metric vs. imperial) is not explicitly covered, which can lead to inconsistency in measurement suggestions.', 'Output format instructions are detailed but could be organized under clearer subheadings, avoiding overlap between Instructions and Rules sections.', 'No explicit instruction to ask follow-up questions beyond ingredient availability; model might proceed with assumptions in other ambiguous scenarios.', 'The persona is defined briefly but could benefit from a more explicit Role and Objective statement upfront.', 'The prompt does not instruct the model to scale recipes or clarify serving sizes, which are common user needs.'],

In [34]:
print(res.improved_prompt)

# Role and Objective
You are a friendly, creative, and precise culinary assistant. Your goal is to help users discover or create easy-to-follow, delicious, and safe recipes tailored to their preferences, equipment, and dietary needs.

# Clarification Guidelines
- Always ask a follow-up question if any key requirement is missing (e.g., serving size, available ingredients, dietary restrictions, cooking equipment).
- Confirm any allergy or dietary preference before finalizing a recipe.

# Response Rules
- Provide a single recipe unless the user explicitly requests multiple options.
- Present ingredient lists with precise measurements using standard units (grams, cups, teaspoons, tablespoons). Offer metric–imperial conversions when helpful.
- When suggesting substitutions or variations, clearly label them (e.g., “Substitution:” or “Variation:”) and note any flavor or texture differences.
- Exclude any ingredients the user is allergic to or has asked to avoid; double-check the final recipe.

In [35]:
res.critique

['Section headings are inconsistent and could be more structured to improve readability and parsing by the model.',
 'Clarification strategy is only applied when ingredients are missing; it should more generally prompt for missing context (dietary restrictions, serving size, equipment) when needed.',
 'Unit conversion guidance (metric vs. imperial) is not explicitly covered, which can lead to inconsistency in measurement suggestions.',
 'Output format instructions are detailed but could be organized under clearer subheadings, avoiding overlap between Instructions and Rules sections.',
 'No explicit instruction to ask follow-up questions beyond ingredient availability; model might proceed with assumptions in other ambiguous scenarios.',
 'The persona is defined briefly but could benefit from a more explicit Role and Objective statement upfront.',
 'The prompt does not instruct the model to scale recipes or clarify serving sizes, which are common user needs.']

In [36]:
res.rationale

['Reorganized into clearly labeled sections (Role and Objective, Clarification, Response Rules, Output Format, Example) to enhance structure and parsing.',
 'Expanded clarification guidelines to cover multiple ambiguous scenarios beyond just missing ingredients (serving size, equipment, dietary preferences).',
 'Added explicit instructions on unit consistency and metric–imperial conversions to standardize measurements.',
 'Consolidated Rules and Instructions into a single Response Rules section to eliminate redundancy and improve clarity.',
 'Included guidance for scaling recipes and nested substitutions/variations to handle common user requests more robustly.',
 'Reinforced safety and ethical considerations with a concise polite decline strategy to prevent background moralizing.',
 'Provided a fully formatted example that demonstrates all requested features (substitutions, scaling notes, precise units).']